In [6]:
# Step 1: Model Loading

import torch
from torchvision import models, transforms
import torch.nn as nn

# Load your trained model architecture
model = models.resnet18(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)  # 3 classes: Normal, Hemorrhagic, Ischemic

# Load the trained weights
model.load_state_dict(torch.load("best_brain_ct_model.pth"))

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set model to evaluation mode
model.eval()

# Class names
class_names = ['Normal', 'Hemorrhagic', 'Ischemic']

In [7]:
# Step 2: Image Processing

from PIL import Image
import numpy as np

# Define the same transformations you used for validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ensure the size matches training
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [8]:
# Step 3: Prediction Function

def predict_image(image_path, model, transform, device):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    # Predict
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = outputs.max(1)
        predicted_class = class_names[predicted.item()]
    
    return predicted_class

In [12]:
# Step 4: Example Usage

# Example: Provide your image path here
image_path = "test2.png"

predicted_label = predict_image(image_path, model, transform, device)
print(f"The model predicts: {predicted_label}")

The model predicts: Ischemic
